<a href="https://colab.research.google.com/github/Jayanth20/Data-Visualization/blob/main/Assignment_1_Jayanth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1: Dino Fun World

### Assignment Description

You, in your role as a data explorer and visualizer, have been asked by the administrators of a small amusement park in your hometown to answer a few questions about their park operations. The dataset that they provided for you to perform the requested analysis includes the movement and communication data captured from the park attendees' apps during one weekend (Friday, Saturday, and Sunday).

The administrators would like you to answer four relatively simple questions about the park activities on the day in question. These questions all deal with park operations and can be answered using the data provided.
* **Question 1:** What is the most popular attraction to visit in the park?


* **Question 2:** What ride (note that not all attractions are rides) has the longest average visit time?


* **Question 3:** Which Fast Food offering has the fewest visitors?


* **Question 4:** Compute the Skyline of number of visits and visit time for the park's ride, and report the rides that appear in the Skyline. (Note: Your answer should be three points, which can be given in any order.)


### Directions

The database provided by the park administration is formatted to be readable by any SQL database library. The course staff recommends the sqlite3 library. The database contains three tables, named 'checkin', 'attractions', and 'sequences'. The database file is named 'dinofunworld.db' .

The information contained in each of these tables is listed below:

`checkin`:
    - The check-in data for all visitors for the day in the park. The data includes two types of check-ins: inferred and actual checkins.
    - Fields: visitorID, timestamp, attraction, duration, type

`attraction`:
    - The attractions in the park by their corresponding AttractionID, Name, Region, Category, and type. Regions are from the VAST Challenge map such as Coaster Alley, Tundra Land, etc. Categories include Thrill rides, Kiddie Rides, etc. Type is broken into Outdoor Coaster, Other Ride, Carousel, etc.
    - Fields: AttractionID, Name, Region, Category, type
    
`sequences`:
    - The check-in sequences of visitors. These sequences list the position of each visitor to the park every five minutes. If the visitor has not entered the part yet, the sequence has a value of 0 for that time interval. If the visitor is in the park, the sequence lists the attraction they have most recently checked in to until they check in to a new one or leave the park.
    - Fields: visitorID, sequence
    
Using the provided data, answer the four questions that the administrators have asked.

### Submission Directions for Assignment Deliverables

This assignment will be auto-graded. We recommend that you submit this assignment. In order for your answers to be correctly registered in the system, you must place the code for your answers in the cell indicated for each question. In addition, you should submit the assignment with the output of the code in the cell's display area. The display area should contain only your answer to the question with no extraneous information, or else the answer may not be picked up correctly. 

Each cell that is going to be graded has a set of comment lines at the beginning of the cell. These lines are extremely important and must not be modified or removed. 

Please execute each cell in Jupyter Notebook before submitting.




In [ ]:
import sqlite3
from collections import Counter
import pandas as pd
from datetime import timedelta
import operator


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
db_path = '/content/gdrive/MyDrive/DV/dinofunworld.db'

### Question 1: 
What is the most popular attraction to visit in the park?

**Note:** Your output should be the name of the attraction.

In [ ]:
# your code here for Q1
import sqlite3
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

query = "SELECT name, COUNT(*) as count FROM checkin INNER JOIN attraction ON checkin.attraction = attraction.AttractionID GROUP BY attraction ORDER BY count DESC LIMIT 1"
cursor.execute(query)
print("The most popular attraction is:", cursor.fetchone()[0])

# conn.close()


The most popular attraction is: Atmosfear


### Question 2: 

What ride (note that not all attractions are rides) has the  longest average visit time?

**Note:** Your output should be the name of the ride.


In [ ]:
# your code here for Q2
query = "SELECT AttractionID, Name FROM attraction where LOWER(Category) LIKE '%ride%';"
cursor.execute(query)
attractions = cursor.fetchall()
time = ("", 0)
for (attr_id, name) in attractions:
    cursor.execute("SELECT duration FROM checkin where attraction="+str(attr_id)+";")
    duration_list = cursor.fetchall()
    total = 0
    count = 0
    for duration in duration_list:
        try:
            d = duration[0].split(':')
            total += timedelta(hours=int(d[0]), minutes=int(d[1]), seconds =int(d[2])).total_seconds()
            count += 1
        except:
            continue
    avgd = total / count
    if avgd > time[1]:
        time = (name, avgd)
print("Ride with Longest visit time is: ",time[0])

Ride with Longest visit time is:  TerrorSaur


### Question 3:

Which Fast Food offering in the park has the fewest visitors?

**Note:** Your output should be the name of the fast food offering.

In [ ]:
# your code here for Q3

query = "SELECT name FROM attraction, checkin WHERE attraction.attractionid = checkin.attraction and attraction.type='Fast Food' GROUP BY name order by count(*) asc limit 1"
cursor.execute(query)
print('Fast Food offering which has the fewest visitors: ', cursor.fetchone()[0])


Fast Food offering which has the fewest visitors:  Theresaur Food Stop


### Question 4:

Compute the Skyline of number of visits and visit time for the park's ride and report the rides that appear in the Skyline. 

**Note:** Remember that in this case, higher visits is better and lower visit times are better. Your output should be formatted as a list of names of the rides in the Skyline. Your output should be three points, which can be given in any order(example output : ['Ride 1','Ride 2','Ride 3'] )


In [ ]:
# your code here for Q4

# Get all the attraction IDs for rides
cursor.execute("SELECT attractionid from attraction WHERE LOWER(category) like LOWER('%ride%')")

rides = {}
for i in cursor.fetchall():
    ride_id = i[0]
    cursor.execute("SELECT duration FROM checkin WHERE attraction=" + str(ride_id))
    
    duration = 0    
    for j in cursor.fetchall():
        visited_count = 0
        try:
            duration += timedelta(int(j[0].split(':')[0]), int(j[0].split(':')[1]), int(j[0].split(':')[2])).total_seconds()
            visited_count += 1
        except:
            pass
    
    rides[ride_id] = [duration]
    
    cursor.execute("SELECT count(*) FROM checkin WHERE attraction=" + str(ride_id))
    num_checkins = cursor.fetchone()[0]
    rides[ride_id].append(num_checkins)
    
visited = list(rides.keys())[0]
# print(rides.keys())
# print(visited)
# print(rides)

# Find the ride with the shortest average duration
# Find the ride with the most check-ins
# Find the ride that is the best balance of short wait time and high popularity
waiting, all = visited, visited

for ride in rides:
    if rides[ride][0] < rides[waiting][0]:
        waiting = ride
    if rides[ride][1] > rides[visited][1]:
        visited = ride
    if (rides[ride][0] < rides[all][0]) and (rides[ride][1] > rides[all][1]):
        all = ride
        
# Get the names of the rides
cursor.execute("SELECT name FROM attraction WHERE attractionid in (" + str(visited) + "," + str(waiting) + "," + str(all) + ")")
attractions = cursor.fetchall()
print([attr[0] for attr in attractions])



['Wrightiraptor Mountain', 'Atmosfear', 'Dykesadactyl Thrill']


In [ ]:
conn.close()